In [1]:
import nest_asyncio
import os
from apsql import AsyncPostgresSaver

nest_asyncio.apply()

In [2]:
DB_NAME: str = "..."
DB_USER: str = "..."
DB_PASSWORD: str = "..."
DB_HOST: str = "..."
DB_PORT: int = ...
os.environ["OPENAI_API_KEY"] = "..."

In [3]:
conn_info = f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} host={DB_HOST} port={DB_PORT}"
memory = AsyncPostgresSaver.from_conn_string(conn_info)

In [4]:
from typing import Annotated, Literal

from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict
class State(TypedDict):
    messages: Annotated[list, add_messages]
@tool
def search(query: str):
    """Call to surf the web."""
    return ["The answer to your question lies within."]
tools = [search]
tool_node = ToolNode(tools)
model = ChatOpenAI(temperature=0, streaming=True)
bound_model = model.bind_tools(tools)
def should_continue(state: State) -> Literal["action", "__end__"]:
    last_message = state["messages"][-1]
    if not last_message.tool_calls:
        return "__end__"
    return "action"
def call_model(state: State):
    response = model.invoke(state["messages"])
    return {"messages": response}
workflow = StateGraph(State)
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("action", "agent")
app = workflow.compile(checkpointer=memory, debug=True)

In [6]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"user_id": "123", "thread_id": "10101"}}
input_message = HumanMessage(content="hi")
await app.ainvoke({"messages": [input_message]}, config)

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='hi')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [HumanMessage(content='hi')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967')]}
[1:tasks] Starting step 1 with 1 task:
- agent -> {'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0')
[1:checkpoint] State at the end of step 1:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
              AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb2

{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0')]}

In [7]:
input_message = HumanMessage(content="my name is juan")
await app.ainvoke({"messages": [input_message]}, config)

[2:checkpoint] State at the end of step 2:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
              AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0')]}
[3:tasks] Starting step 3 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='my name is juan')]}
[3:writes] Finished step 3 with writes to 1 channel:
- messages -> [HumanMessage(content='my name is juan')]
[3:checkpoint] State at the end of step 3:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
              AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0'),
              HumanMessage(content='my name is juan', id='82559867-0b5c-4745-a113-f5ad3ed53fb3')]}
[4:tasks] Starting step 4 with 1 task:
- agent -> {'messages': [HumanMessage(content='hi', id='

{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0'),
  HumanMessage(content='my name is juan', id='82559867-0b5c-4745-a113-f5ad3ed53fb3'),
  AIMessage(content='Nice to meet you, Juan! How can I help you today?', response_metadata={'finish_reason': 'stop'}, id='run-4b53211c-13c5-49fb-b613-767e449555c8-0')]}

In [8]:
input_message = HumanMessage(content="what is my name?")
await app.ainvoke({"messages": [input_message]}, config)

[5:checkpoint] State at the end of step 5:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
              AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0'),
              HumanMessage(content='my name is juan', id='82559867-0b5c-4745-a113-f5ad3ed53fb3'),
              AIMessage(content='Nice to meet you, Juan! How can I help you today?', response_metadata={'finish_reason': 'stop'}, id='run-4b53211c-13c5-49fb-b613-767e449555c8-0')]}
[6:tasks] Starting step 6 with 1 task:
- __start__ -> {'messages': [HumanMessage(content='what is my name?')]}
[6:writes] Finished step 6 with writes to 1 channel:
- messages -> [HumanMessage(content='what is my name?')]
[6:checkpoint] State at the end of step 6:
{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
              AIMessage(content='Hello! How can I assist you today?', response_

{'messages': [HumanMessage(content='hi', id='03ee94c8-e10a-4a16-a188-e19b83a60967'),
  AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop'}, id='run-d4ba6652-bb27-443d-9c31-43b7680ea508-0'),
  HumanMessage(content='my name is juan', id='82559867-0b5c-4745-a113-f5ad3ed53fb3'),
  AIMessage(content='Nice to meet you, Juan! How can I help you today?', response_metadata={'finish_reason': 'stop'}, id='run-4b53211c-13c5-49fb-b613-767e449555c8-0'),
  HumanMessage(content='what is my name?', id='b9584b17-30de-4fd5-8fed-6924baebb12a'),
  AIMessage(content='Your name is Juan.', response_metadata={'finish_reason': 'stop'}, id='run-8805677e-69f0-4d65-8aff-64813f7a305d-0')]}